# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 31 2022 10:56AM,245415,21,625955,"NBTY Global, Inc.",Released
1,Aug 31 2022 10:56AM,245425,10,MSP213579,"Methapharm, Inc.",Released
2,Aug 31 2022 10:56AM,245425,10,MSP213580,"Methapharm, Inc.",Released
3,Aug 31 2022 10:56AM,245425,10,MSP213582,"Methapharm, Inc.",Released
4,Aug 31 2022 10:56AM,245425,10,MSP213586,"Methapharm, Inc.",Released
5,Aug 31 2022 10:56AM,245425,10,MSP213588,"Methapharm, Inc.",Released
6,Aug 31 2022 10:56AM,245425,10,MSP213590,"Methapharm, Inc.",Released
7,Aug 31 2022 10:56AM,245424,10,MSP213568,"Methapharm, Inc.",Released
8,Aug 31 2022 10:56AM,245424,10,MSP213569,"Methapharm, Inc.",Released
9,Aug 31 2022 10:56AM,245424,10,MSP213570,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,245419,Released,29
21,245421,Released,6
22,245423,Released,5
23,245424,Released,22
24,245425,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245419,NaN,1.0,29.0
245421,NaN,NaN,6.0
245423,NaN,NaN,5.0
245424,NaN,NaN,22.0
245425,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,0.0,0.0,66.0
245276,0.0,0.0,1.0
245277,0.0,1.0,0.0
245281,0.0,0.0,1.0
245282,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244946,0,0,66
245276,0,0,1
245277,0,1,0
245281,0,0,1
245282,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,0,0,66
1,245276,0,0,1
2,245277,0,1,0
3,245281,0,0,1
4,245282,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244946,,,66
1,245276,,,1
2,245277,,1,
3,245281,,,1
4,245282,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc."
1,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc."
7,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc."
29,Aug 31 2022 10:55AM,245414,21,"NBTY Global, Inc."
30,Aug 31 2022 10:47AM,245423,10,ISDIN Corporation
35,Aug 31 2022 10:46AM,245421,10,Eywa Pharma Inc.
41,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc."
71,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc."
72,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.
94,Aug 31 2022 9:56AM,245413,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc.",,,1
1,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc.",,,6
2,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc.",,,22
3,Aug 31 2022 10:55AM,245414,21,"NBTY Global, Inc.",,,1
4,Aug 31 2022 10:47AM,245423,10,ISDIN Corporation,,,5
5,Aug 31 2022 10:46AM,245421,10,Eywa Pharma Inc.,,,6
6,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc.",,1,29
7,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc.",,1,
8,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.,,4,18
9,Aug 31 2022 9:56AM,245413,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc.",1,,
1,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc.",6,,
2,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc.",22,,
3,Aug 31 2022 10:55AM,245414,21,"NBTY Global, Inc.",1,,
4,Aug 31 2022 10:47AM,245423,10,ISDIN Corporation,5,,
5,Aug 31 2022 10:46AM,245421,10,Eywa Pharma Inc.,6,,
6,Aug 31 2022 10:30AM,245419,15,"Alliance Pharma, Inc.",29,1,
7,Aug 31 2022 10:08AM,245416,21,"NBTY Global, Inc.",,1,
8,Aug 31 2022 9:59AM,245403,20,Alumier Labs Inc.,18,4,
9,Aug 31 2022 9:56AM,245413,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc.",1,,
1,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc.",6,,
2,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc.",22,,
3,Aug 31 2022 10:55AM,245414,21,"NBTY Global, Inc.",1,,
4,Aug 31 2022 10:47AM,245423,10,ISDIN Corporation,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc.",6.0,NaN,NaN
2,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc.",22.0,NaN,NaN
3,Aug 31 2022 10:55AM,245414,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Aug 31 2022 10:47AM,245423,10,ISDIN Corporation,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc.",6.0,0.0,0.0
2,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc.",22.0,0.0,0.0
3,Aug 31 2022 10:55AM,245414,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Aug 31 2022 10:47AM,245423,10,ISDIN Corporation,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1717904,49.0,11.0,0.0
15,735701,107.0,1.0,34.0
20,490742,47.0,4.0,0.0
21,2207931,7.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1717904,49.0,11.0,0.0
1,15,735701,107.0,1.0,34.0
2,20,490742,47.0,4.0,0.0
3,21,2207931,7.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,49.0,11.0,0.0
1,15,107.0,1.0,34.0
2,20,47.0,4.0,0.0
3,21,7.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,15,Released,107.0
2,20,Released,47.0
3,21,Released,7.0
4,10,Executing,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,20,21
Status,,,,
Completed,0.0,34.0,0.0,0.0
Executing,11.0,1.0,4.0,2.0
Released,49.0,107.0,47.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,20,21
0,Completed,0.0,34.0,0.0,0.0
1,Executing,11.0,1.0,4.0,2.0
2,Released,49.0,107.0,47.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,20,21
0,Completed,0.0,34.0,0.0,0.0
1,Executing,11.0,1.0,4.0,2.0
2,Released,49.0,107.0,47.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()